# Designing Typeahead Suggestion

Typeahead is a real-time suggestion service which recommends terms to users as they enter text for searching.

As the user types into the search box, it tries to predict the query based on the characters the users has entered, and gives a list of suggestions to complete the query.

It's not about speeding up the users' search but to help the user articulate their search queries better.


## 1. Requirements and Goals of the System
**Functional requirements:** As the user types in their search query, our service should suggest top 10 terms starting with whatever the user typed.

**Non-functional requirements:** The suggestions should appear in real-time, allowing the user to see it in about 200ms.


# 2. Basic System Design and Algorithm

The problem to solve is that we have a lot of strings we need to store in such a way that the user can search with any prefix. The service will suggest the terms that match with the prefix. For example, if our DB contains the terms (cat, cap, captain, capital), and the user has typed in `cap`, then the system should suggest `cap`, `captain` and `capital`.

To serve a lot of queries with minimal latency, we can't depend on the DB for this; we need to store our index in memory in a highly efficient data structure – a Trie(pronounced "try").

![](images/trie_typeahead.png)

If the user types `cap`, then the service can traverse the trie and go to the node **P**, to find all the terms that start with this prefix. (i.e cap-ital, cap-tain, cap-tion).

We can also merge nodes that have only one branch to save memory.

![](images/trie_merged.png)


**Should we have a case insensitive trie?** For simplicity, lets assume our data is case insensitive.

#### How do we find top 10 suggestions?
We can store the count of searches that terminated at each node. For example, if the user about `captain` 20 times, and `caption` 50 times, we can store the count with the last character of the phrase. Now if the user types `cap` we know that the top most searched word under 'cap' is `caption`.

> So to find the top suggestions for a given prefix, we can traverse the sub-tree under it.

#### Given a prefix, how long will it take to traverse its sub-tree?
Given the amounf of text we need to index, we should expect a huge tree. Traversing a tree will take really long. Since we have strict latency requrements, we need to improve the efficiency of our solution.
- Store top 10 suggestions for each node. This will require extra storage.
- We can optimize our storage by storing only references to the terminal node rather than storing the entire phrase. 
- Store frequency with each reference to keep track of top suggestions.

#### How do we build this trie?
We can efficiently build it bottom-up. 
- Each parent node will recursively call child nodes to calculate top suggestions and their counts.
- The parent nodes will combine their suggestions from all their children nodes to determine their top suggestions.

#### how do we update the trie?
Assume 5 billion daily searches
```
5B / 36400 ~= 60000 searches/sec
```
Updating the trie for every search will be extremely resource intensive and this can hamper our read requests too.
Solution: Update the trie offline at certain intervals.

As new queries come in, log them and track their frequency of occurences. We can log every 1000th query. For example, if we don't want to show a term that hasn't been searched for < 1000 times, it's safe to only log queries that occur the 1000th time.

We can user [Map Reduce (MR)](https://en.wikipedia.org/wiki/MapReduce) to process all the logging data periodically, say every hour. 
- The MR jobs will calculate the frequencies of all search terms in the past hour.
- Then update the trie with the new data. We take the current snapshot of the trie and update it offline with the new terms and their frequencies.

We have two options to updating offline:
1. Make a copy of the trie from each server and update the copy offline. Once done, switch to the copy and discard the old one.
2. Master-slave configure each trie server. Update slave while master is service incoming traffic. Once update is complete, make our slave the new master. Then later, update our old master, which can then start serving traffic too.

